In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import os



# ================================================================== #
#                     选择特征提取器函数
#     使用SIFT特征，借助CV2中的函数提取特征，也可修改为其他特征提取方式
# ================================================================== #
def detectAndDescribe(image):
    # SIFT特征提取器
    descriptor = cv2.SIFT_create()
    # 若使用其他特征，以BRISK特征为例，按下面的格式定义新的特征提取器
    # descriptor = cv2.BRISK_create()
    (kps, features) = descriptor.detectAndCompute(image, None)
    return (kps, features)


# ================================================================== #
#                     使用knn检测函数
#           SIFT特征建议使用欧几里得距离，而BRISK特征建议使用汉明距离
# ================================================================== #
def matchKeyPointsKNN(featuresA, featuresB, ratio):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    # BRISK特征
    # bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    rawMatches = bf.knnMatch(featuresA, featuresB, 2)
    matches = []
    for m, n in rawMatches:
        if m.distance < n.distance * ratio:
            matches.append(m)
    return matches


# ================================================================== #
#                     计算关键点的透视关系
# ================================================================== #
def getHomography(kpsA, kpsB, matches, reprojThresh):
    kpsA = np.float32([kp.pt for kp in kpsA])
    kpsB = np.float32([kp.pt for kp in kpsB])
    if len(matches) > 4:

        ptsA = np.float32([kpsA[m.queryIdx] for m in matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in matches])

        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
                                         reprojThresh)
        return (matches, H, status)
    else:
        return None


# ================================================================== #
#                     去除图像黑边
# ================================================================== #
def cutBlack(pic,mode=0):
    rows, cols = np.where(pic[:, :, 0] != 0)
    min_row, max_row = min(rows), max(rows) + 1
    min_col, max_col = min(cols), max(cols) + 1
    pic =pic[min_row:max_row, min_col:max_col, :]
    #设置mode=1则进行全部黑边的删除，保证图像内没有黑边
    if mode==1:
        gray = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

        for row in range(thresh.shape[0]):
            if thresh[row, 0] == 0:
                break
        first_nonzero_row = row

        for col in range(thresh.shape[1]):
            if thresh[0, col] == 0:
                break
        first_nonzero_col = col

        pic = pic[0:first_nonzero_row, 0:first_nonzero_col, :]
    return pic


# ================================================================== #
#                          调换
# ================================================================== #
def swap(a, b):
    return b, a


# ================================================================== #
#                            主要的函数
#   默认使用SIFT特征，修改为其他特征时注意修改detectAndDescribe函数中的特征提取器
#          和matchKeyPointsKNN函数中的距离计算，以达到好的效果
# ================================================================== #
def handle(path1, path2, isShow=False):
    """
    读取原始图像
    """
    if isinstance(path2, str):
        imageA = cv2.imread(path2)
        imageA = cv2.cvtColor(imageA, cv2.COLOR_BGR2RGB)
    else:
        imageA = path2
    imageA_gray = cv2.cvtColor(imageA, cv2.COLOR_RGB2GRAY)

    if isinstance(path1, str):
        imageB = cv2.imread(path1)
        imageB = cv2.cvtColor(imageB, cv2.COLOR_BGR2RGB)
    else:
        imageB = path1
    imageB_gray = cv2.cvtColor(imageB, cv2.COLOR_RGB2GRAY)

    """
    显示输入的两张图片
    """
    if isShow:
        f = plt.figure(figsize=(10, 4))
        f.add_subplot(1, 2, 1)
        plt.title("imageB")
        plt.imshow(imageB)
        plt.xticks([]), plt.yticks([])
        f.add_subplot(1, 2, 2)
        plt.title("imageA")
        plt.imshow(imageA)
        plt.xticks([]), plt.yticks([])
    """
    提取两张图片的特征
    """
    kpsA, featuresA = detectAndDescribe(imageA_gray)
    kpsB, featuresB = detectAndDescribe(imageB_gray)
    """
    显示关键点
    """
    if isShow:
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), constrained_layout=False)
        ax1.imshow(cv2.drawKeypoints(imageA_gray, kpsA, None, color=(0, 255, 0)))
        ax1.set_xlabel("(a)key point", fontsize=14)
        ax2.imshow(cv2.drawKeypoints(imageB_gray, kpsB, None, color=(0, 255, 0)))
        ax2.set_xlabel("(b)key point", fontsize=14)

    """
    进行特征匹配
    """
    matches = matchKeyPointsKNN(featuresA, featuresB, ratio=0.75)
    if len(matches) < 10:
        return None
    img3 = cv2.drawMatches(imageA, kpsA, imageB, kpsB, np.random.choice(matches, 100),
                           None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    """
    匹配的特征展示
    """
    if isShow:
        fig = plt.figure(figsize=(10, 4))
        plt.imshow(img3)
        plt.title("feature match")
        plt.axis('off')

    """
    计算两张图的透视关系
    """
    matchCount = len(matches)
    M = getHomography(kpsA, kpsB, matches, reprojThresh=4)
    if M is None:
        print("Error!")
    (matches, H, status) = M
    """
    将图片A进行透视变换并检查图片位置
    """
    result = cv2.warpPerspective(imageA, H,
                                 ((imageA.shape[1] + imageB.shape[1]) * 2, (imageA.shape[0] + imageB.shape[0]) * 2))
    resultAfterCut = cutBlack(result)

    # 检查图片位置
    if np.size(resultAfterCut) < np.size(imageA) * 0.95:
        print("图片位置不对,将自动调换")
        # 调换图片
        kpsA, kpsB = swap(kpsA, kpsB)
        imageA, imageB = swap(imageA, imageB)

        matches = matchKeyPointsKNN(featuresB, featuresA, ratio=0.75)
        if len(matches) < 10:
            return None
        matchCount = len(matches)
        M = getHomography(kpsA, kpsB, matches, reprojThresh=4)
        if M is None:
            print("Error!")
        (matches, H, status) = M
        result = cv2.warpPerspective(imageA, H,
                                     ((imageA.shape[1] + imageB.shape[1]) * 2, (imageA.shape[0] + imageB.shape[0]) * 2))
    """
    合并图片
    """
    result[0:imageB.shape[0], 0:imageB.shape[1]] = np.maximum(imageB, result[0:imageB.shape[0], 0:imageB.shape[1]])
    #去掉全部黑边，是图像更美观，但会损失图像信息
    result = cutBlack(result,mode=1)
    return result, matchCount


# ================================================================== #
#                     合并多张图
# ================================================================== #
def handleMulti(img_paths, isShow=False):
    # print(isShow)
    l = len(img_paths)
    if isShow:
        row = math.ceil(l / 3)

        f = plt.figure(figsize=(10, 4))
        for i in range(l):
            f.add_subplot(row, 3, i + 1)
            plt.title(f"image({i + 1})")
            plt.axis("off")
            plt.imshow(cv2.cvtColor(cv2.imread(img_paths[i]), cv2.COLOR_BGR2RGB))
    assert (l > 1)
    pic_num = 1 #拼接图的数量
    isHandle = [0 for i in range(l - 1)] #被拼接的标志位
    nowPic = img_paths[0]
    img_paths = img_paths[1:]
    for j in range(l - 1):
        isHas = False  # 在一轮中是否找到
        matchCountList = [] #特征点匹配数
        resultList = [] #拼接结果
        indexList = [] #保存被拼接图像的下标
        for i in range(l - 1):
            if (isHandle[i] == 1):
                continue
            result, matchCount = handle(nowPic, img_paths[i])
            if not result is None:
                matchCountList.append(matchCount)
                resultList.append(result)
                indexList.append(i)
                isHas = True
        if not isHas:  # 一轮找完都没有可以合并的
            if pic_num==1:
                return None
            else:
                return nowPic
        else:
            index = matchCountList.index(max(matchCountList))
            nowPic = resultList[index]
            pic_num+=1
            isHandle[indexList[index]] = 1
           
    return nowPic


# ================================================================== #
#                     主函数
# ================================================================== #
if __name__ == "__main__":
    # 在这两个地址中已经存在拼接好的图像，运行前删除拼接好的图像
    # img_dir = 'img/image_multi'  # 多个待拼接图片的文件地址
    img_dir = 'img/image_two' #两个待拼接图片的文件地址

    img_paths = []
    for img_name in os.listdir(img_dir):
        img_paths.append(os.path.join(img_dir, img_name))

    print(img_paths)
    if len(img_paths) > 2:
        result = handleMulti(img_paths, isShow=False)

    elif len(img_paths) == 2:
        result, _ = handle(img_paths[0], img_paths[1], isShow=False)
    else:
        print('error:图像拼接最少需要两张图像')
        exit()

    if not result is None:
        cv2.imshow("result", result[:, :, [2, 1, 0]])
        cv2.imwrite(os.path.join(img_dir, 'spliced_image.jpg'), result[:, :, [2, 1, 0]])
        plt.show()
        cv2.waitKey(0)
    else:
        print("没有找到对应特征点,无法合并")
    exit()


['img/image_multi\\1.jpg', 'img/image_multi\\2.jpg', 'img/image_multi\\3.jpg']
